# Imports
---
Import new skis / snowboards to an SQL database

In [1]:
import json
import pymysql
import re

ModuleNotFoundError: No module named 'pymysql'

In [9]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../app/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

In [10]:
db = setupdb()
cursor = db.cursor()

In [76]:
params = {
    'size': {'aliases': ['board size', 'size', 'length'], 'found': False, 'keep': True, 'values': []},
    'binding size': {'aliases': ['binding sizes', 'binding size'], 'found': False, 'keep': False, 'values': []},
    'running length': {'aliases': ['running length'], 'found': False, 'keep': True, 'values': []},
    'nose width': {'aliases': ['nose width', 'tip width'], 'found': False, 'keep': True, 'values': []},
    'waist width': {'aliases': ['waist width'], 'found': False, 'keep': True, 'values': []},
    'tail width': {'aliases': ['tail width'], 'found': False, 'keep': True, 'values': []},
    'sidecut depth': {'aliases': ['sidecut depth'], 'found': False, 'keep': False, 'values': []},
    'sidecut': {'aliases': ['sidecut radius', 'turning radius', 'radius', 'sidecut'], 'found': False, 'keep': True, 'values': []},
    'setback': {'aliases': ['stance setback', 'stance location', 'setback'], 'found': False, 'keep': True, 'values': []},
    'effective edge': {'aliases': ['effective edge'], 'found': False, 'keep': True, 'values': []},
    'weight range': {'aliases': ['weight range'], 'found': False, 'keep': False, 'values': []},
    'reference stance': {'aliases': ['stance width', 'reference stance'], 'found': False, 'keep': True, 'values': []}
}

In [77]:
values = {
    'size': '',
    'nose width': '',
    'waist width': '',
    'tail width': '',
    'sidecut': '',
    'setback': '',
    'effective edge': ''
}

In [82]:
# Extraction of data 
# from string format to a tabular structure
# --------------------------------------------------
def extract_data(table):
    # Remove units from headings
    while re.search(r'\(([a-zA-Z]+)\)', table) != None:
        match = re.search(r'\(([a-zA-Z]+)\)', table)
        substr = table[match.start():match.end()]
        table = table.replace(substr, '')
    '''    
    while ')' in table:
        table = table[:table.find('(')] + table[table.find(')') + 1:]
    '''    
        
    breakpoints = []
    
    # Find location of headings in table
    for key, param in params.items():
        # Check each param alias
        print(param)
        for alias in param['aliases']:
            print(f"Looking for {alias}")
            if alias in table:
                breakpoints.append(table.find(alias))
                break
        
        
    # Extract table data into rows
    rows = []
    breakpoints.sort()
    for i, b in enumerate(breakpoints):
        if i > 0:
            rows.append(table[breakpoints[i-1]:b])
            print(f"Row: {table[breakpoints[i-1]:b]}")
        if i == len(breakpoints) -1:
            rows.append(table[b:])
    
    
    # Remove lables from rows
    for row in rows:
        for key, param in params.items():
            for alias in param['aliases']:
                if alias in row and not param['found']:
                    vals = row.replace(alias, '')
                    vals = vals.strip()
                    vals = vals.replace('\u0020\u002f\u0020', '/')
                    vals = vals.replace('\u0020\u200b\u002f\u0020', '/')
                    vals = vals.replace('\u0020\u200b\u200b\u002f\u0020', '/')
                    vals = vals.split() 
                    param['values'] = vals
                    param['found'] = True
                    break

    '''
    # Format table rows as lists
    for val in values:
        try:
            # \u0020\u200b\u002f\u0020
            # \u0020\u002f\u0020
            values[val] = values[val].strip()
            values[val] = values[val].replace('\u0020\u002f\u0020', '/')
            values[val] = values[val].replace('\u0020\u200b\u002f\u0020', '/')
            values[val] = values[val].replace('\u0020\u200b\u200b\u002f\u0020', '/')
            values[val] = values[val].split() 
            
        except Exception as e:
            x = e
            '''
    
    # Truncate long rows to the correct size
    num_sizes = len(params['size']['values'])
    for key, param in params.items():
        # Sidecuts can have more than one radius
        if param != "sidecut":
            param['values'] = param['values'][:num_sizes]

        # Remove rogue characters from row values
        if len(param['values']) > 0 and type(param['values'][0]) == str:
            for x, v in enumerate(param['values']):
                param['values'][x] = param['values'][x].replace('\u200b', '')
        
        
        # Fill empty or missing data with null values
        for x in range(num_sizes - len(param['values'])):
            
            param['values'].append(0)
            
    
        
        
    return params

In [14]:
# Save skiboard in MySQL DB
# --------------------------------------------------
def save_skiboard(skiboard):
    try:
        sql = f"""INSERT INTO skiboards (
            url,
            brand,
            model,
            year,
            name,
            slug,
            category,
            family,
            description,
            stiffness,
            shape,
            flex_profile,
            camber_profile,
            camber_details,
            core,
            core_profiling,
            fibreglass,
            laminates,
            resin,
            base,
            edges,
            edge_tech,
            topsheet,
            sidewall,
            inserts,
            asym,
            weight,
            womens,
            youth
        ) VALUES (
            '{skiboard['url']}',
            '{skiboard['brand']}',
            '{skiboard['model']}',
            '{skiboard['year']}',
            '{str(skiboard['brand']).title()} {str(skiboard['model']).title()} {str(skiboard['year'])}',
            '{str(skiboard['brand']).lower().replace(' ', '-')}-{str(skiboard['model']).lower().replace(' ', '-')}-{str(skiboard['year'])}',
            '{skiboard['category']}',
            '{skiboard['family']}',
            '{skiboard['description']}',
            '{skiboard['stiffness']}',
            '{skiboard['shape']}',
            '{skiboard['flex_profile']}',
            '{skiboard['camber_profile']}',
            '{skiboard['camber_details']}',
            '{skiboard['core']}',
            '{skiboard['core_profiling']}',
            '{skiboard['fibreglass']}',
            '{skiboard['laminates']}',
            '{skiboard['resin']}',
            '{skiboard['base']}',
            '{skiboard['edges']}',
            '{skiboard['edge_tech']}',
            '{skiboard['topsheet']}',
            '{skiboard['sidewall']}',
            '{skiboard['inserts']}',
            '{skiboard['asym']}',
            '{skiboard['weight']}',
            '{skiboard['womens']}',
            '{skiboard['youth']}'
        )"""
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(f"ERROR:\n{e}")

In [15]:
# Save data in MySQL DB
# --------------------------------------------------
def save_values(skiboard, values):
    #print(f"Importing...\n{str(skiboard['brand']).capitalize()} {str(skiboard['model']).capitalize()} {str(skiboard['year'])}\nValues:\n{values}")

    for x in range(len(values['size'])):
        try:
            sql = f"""REPLACE INTO sizes (
                skiboard_id,
                size,
                nose_width,
                waist_width,
                tail_width,
                sidecut,
                setback,
                effective_edge
            ) VALUES (
                '{str(skiboard['id'])}',
                '{str(values['size'][x])}',
                '{float(values['nose width'][x])}',
                '{float(values['waist width'][x])}',
                '{float(values['tail width'][x])}',
                '{str(values['sidecut'][x])}',
                '{float(values['setback'][x])}',
                '{float(values['effective edge'][x])}'
            )"""

            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(f"ERROR: size ({values['size'][x]})\n{e}\n")
            print(f"SQL:\n{sql}\n\n")
    
    return True

# Create New SkiBoard
---

In [528]:
id_counter = 180

In [529]:
print(id_counter)

180


In [543]:
skiboard = {
    'id': id_counter,
    'url': '',
    'brand': 'Rome',
    'model': 'Mod',
    'year': 2020,
    'category': 'Snowboard',
    'family': '',
    'description': "Send your boring board to The Hague. The Rome Warden Snowboard rips groomers, polices the tree runs, and liberates you from the cruelty of having to pay for the privilege to rely on someone else's beat-up rental junk. Lay down a mean euro, play in a secret pow stash and slash your way down the hill with authori-tah. The Rome Warden Snowboard hates to be confined and loves to get after it day after day thanks to its durable extruded base and bamboo stringer construction.".replace("'", "`"),
    'stiffness': 8,
    'shape': 'Twin',
    'flex_profile': '',
    'camber_profile': 'Full Camber',
    'camber_details': 'Stay Positive Camber',
    'core': "Pop Core Matrix".replace("'", "`"),
    'core_profiling': '',
    'fibreglass': 'Biax',
    'laminates': 'TurboRod Configuration ~ Glasspack Impact Plates',
    'base': 'SinterCarbon Base',
    'edges': '',
    'edge_tech': '',
    'topsheet': '',
    'resin': '',
    'sidewall': '',
    'inserts': '4x2',
    'asym': 0,
    'weight': 0,
    'womens': 0,
    'youth': 0
}

In [544]:
save_skiboard(skiboard)

In [545]:
print(skiboard['id'])

182


# Add Sizes to SkiBoard
---

In [546]:
#skiboard['id'] = 181

In [28]:
table = input("Please submit yourski/snowboard size table\n\n").lower()

Please submit yourski/snowboard size table

Board Size	149	156	161 Weight Range	100-150 lbs. / 45-68kg	150-200 lbs. / 68-91kg	180-260 lbs.+ / 82-118kg+ Waist Width	243mm	252mm	255mm Stance Location	-25	-25	-25 Binding Sizes	Men's S, Women's M	Men's M/L, Women's L	Men's L, Women's L Running Length	1095mm	1165mm	1215mm Sidecut Radius	7.1m	7.6m	7.9m Sidecut Depth	21.1mm	22.2mm	23.4mm Stance Width	530mm	560mm	560mm Nose Width	285.7mm	297.0mm	302.3mm Tail Width	284.7mm	296.0mm	301.3mm Effective Edge	1135mm	1205mm	1255mm


In [83]:
values = extract_data(table)

{'aliases': ['board size', 'size', 'length'], 'found': False, 'keep': True, 'values': []}
Looking for board size
{'aliases': ['binding sizes', 'binding size'], 'found': False, 'keep': False, 'values': []}
Looking for binding sizes
{'aliases': ['running length'], 'found': False, 'keep': True, 'values': []}
Looking for running length
{'aliases': ['nose width', 'tip width'], 'found': False, 'keep': True, 'values': []}
Looking for nose width
{'aliases': ['waist width'], 'found': False, 'keep': True, 'values': []}
Looking for waist width
{'aliases': ['tail width'], 'found': False, 'keep': True, 'values': []}
Looking for tail width
{'aliases': ['sidecut depth'], 'found': False, 'keep': False, 'values': []}
Looking for sidecut depth
{'aliases': ['sidecut radius', 'turning radius', 'radius', 'sidecut'], 'found': False, 'keep': True, 'values': []}
Looking for sidecut radius
{'aliases': ['stance setback', 'stance location', 'setback'], 'found': False, 'keep': True, 'values': []}
Looking for stan

In [84]:
for row in values:
    print(f"{row}:\n{values[row]}\n\n")

size:
{'aliases': ['board size', 'size', 'length'], 'found': True, 'keep': True, 'values': ['149', '156', '161']}


binding size:
{'aliases': ['binding sizes', 'binding size'], 'found': True, 'keep': False, 'values': ["men's", 's,', "women's"]}


running length:
{'aliases': ['running length'], 'found': True, 'keep': True, 'values': ['1095mm', '1165mm', '1215mm']}


nose width:
{'aliases': ['nose width', 'tip width'], 'found': True, 'keep': True, 'values': ['285.7mm', '297.0mm', '302.3mm']}


waist width:
{'aliases': ['waist width'], 'found': True, 'keep': True, 'values': ['243mm', '252mm', '255mm']}


tail width:
{'aliases': ['tail width'], 'found': True, 'keep': True, 'values': ['284.7mm', '296.0mm', '301.3mm']}


sidecut depth:
{'aliases': ['sidecut depth'], 'found': True, 'keep': False, 'values': ['21.1mm', '22.2mm', '23.4mm']}


sidecut:
{'aliases': ['sidecut radius', 'turning radius', 'radius', 'sidecut'], 'found': True, 'keep': True, 'values': ['7.1m', '7.6m', '7.9m']}


setback:

In [550]:
success = save_values(skiboard, values)
if success:
    print("\nSUCCESS!")
    id_counter += 1


SUCCESS!
